In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import openpyxl
from matplotlib.backends.backend_pdf import PdfPages
import os

In [2]:
# location of files
output_data = r"C:\Users\stephenl\OneDrive - GC International AG\ML\forecast function demo\templates\output_data\\"
sell_out_data_path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\forecast function demo\templates\selloutdata\\'
inventory_path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\forecast function demo\templates\inventory\\'
path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\forecast function demo\templates\input_data\\' #top 20 products

In [3]:
def past6mths(current_date = datetime.today()): #----------------------------------
    n = 7
    past_date = current_date - relativedelta(months=n)
    return past_date.strftime('%B-%Y')

past6mths(), datetime.today()

('July-2022', datetime.datetime(2023, 2, 21, 10, 14, 2, 353471))

# Create function (6 months OTD data)

In [4]:
def forecast6(num):


    func = lambda x: 0 if x < 0 else x
    func1 = lambda x: x if x <0 else 0


    def past5mths(current_date = datetime.today()): #----------------------------------
        n = 4
        past_date = current_date - relativedelta(months=n)
        return past_date.strftime('%B-%Y')

    def past6mths(current_date = datetime.today()): #----------------------------------
        n = 6
        past_date = current_date - relativedelta(months=n)
        return past_date.strftime('%B-%Y')

    def future10mths(current_date = datetime.today()): #----------------------------------
        n = 10
        future_date = current_date + relativedelta(months=n)
        return future_date.strftime('%B-%Y')

    def today(current_date = datetime.today()):
        return current_date.strftime('%B-%Y')

    def nextmth(current_date = datetime.today()): #----------------------------------
        n = 1
        future_date = current_date + relativedelta(months=n)
        return future_date.strftime('%B-%Y')
    
    def lastmonth():
        n=-1
        a =  datetime.today()+relativedelta(months=n)
        return a.strftime('%B-%Y')
    
    def future6mths(current_date = datetime.today()): #----------------------------------
        n = 5
        future_date = current_date + relativedelta(months=n)
        return future_date.strftime('%B-%Y')


    start = past6mths()
    end = future10mths()
    this_mth = today()
    next_mth = nextmth()
    last_mth = lastmonth()
    end_of_forecast = future6mths()


    def startfunction(a_=0,b_=0,c_=0, d_=0, e_=0):

        global df

        # create empty dataframe and date range

        d=pd.date_range(start=past6mths(), end=future10mths(), freq='MS')
        df=pd.DataFrame(columns=['date','cls_stk_dpp','open_ord','total','fcast_open_stk','fcast_gds_rec','otd','otd_fcast','recom_topup',
                                'recom_safe_stk','fcast_cls_stk','recom_ord','rssl_less_stk',
                                'next_mth_cls_stk'])

        # fill the dates with date range

        df.date=d
        df = df.fillna(0)

        # system input order for 'cls_stk_dpp' & 'open_ord' and 'total'

        df.loc[6,'cls_stk_dpp']=cls_stk_dpp

        df.loc[6,'open_ord']=open_ord_dpp

        df.total=df.cls_stk_dpp + df.open_ord

        # input otd data from row 0 to row 5

        y=[first,second,third,fouth,fifth, sixth]
        df.loc[:5,'otd']=y

        # convert dates to ordinal values

        df['ord_date'] = [i.toordinal() for i in df.date]

        # fill column with ordinal dates as independent training data for linear regression

        otd_date = df.loc[0:5,'ord_date']

        # run linera regression to produce m and b

        m, b, rv, p, se = stats.linregress(otd_date,y)

        # prepare ordinal dates for prediction and convert to list

        fcast_date = df.loc[6:11,'ord_date']
        fcast_date = fcast_date.tolist()

        # forecast prediction using ordinal dates

        fcast = [i*m+b for i in fcast_date]

        fcast = [i.round(0) for i in fcast]

        fcast = [func(i) for i in fcast]

        # fill column with predicted values in 'otd_fcast'

        df.loc[6:11,'otd_fcast']=fcast

        # recommended top up qty

        df.loc[6,'recom_topup']=topup



        # recommended safety stocks

        safe_stks = round(df.loc[0:5,'otd'].mean()*leadtime,0)
        df.loc[6,'recom_safe_stk']=safe_stks


        def fillrows(h):
        # forecasted closing stock (O)

            s = 1

            if h >= 7:

                df.loc[h,'fcast_cls_stk'] = df.loc[h,'fcast_open_stk'] + df.loc[h,'fcast_gds_rec'] - df.loc[h,'otd_fcast'] + \
                func1(df.loc[h-1,'fcast_cls_stk'])-df.loc[h-1,'recom_topup']

            else:

                df.loc[h,'fcast_cls_stk']= df.loc[h,'cls_stk_dpp']-df.loc[h,'otd_fcast']


            # forecasted opening stock (G)

            df.loc[h+1,'fcast_open_stk']=func(df.loc[h,'fcast_cls_stk'])


            # recommended order qty (p)

            if h>= 7:

                b = df.loc[h,'otd_fcast'] + df.loc[h,'recom_topup'] + df.loc[h,'recom_safe_stk'] - df.loc[h,'fcast_open_stk']
                c = func(b)
                df.loc[h,'recom_ord'] = c + df.loc[h+s,'rssl_less_stk'] # change to s -------------------------------------------------
                
                
            else:

                b = df.loc[h,'otd_fcast'] + df.loc[h,'recom_topup'] + df.loc[h,'recom_safe_stk'] - df.loc[h,'total']
                c = func(b)
                df.loc[h,'recom_ord'] = c + df.loc[h+s,'rssl_less_stk'] # change to s -------------------------------------------------


            # forecasted goods received (H)

            df.loc[h+s,'fcast_gds_rec'] = df.loc[h,'recom_ord'] + df.loc[h,'open_ord'] # change to s -----------------------------------------
            


#             df.loc[h+s,'fcast_gds_rec'] = df.loc[h,'recom_ord']
            # next month closing stock (S)

            df.loc[h,'next_mth_cls_stk']=  safe_stks - df.loc[h,'fcast_cls_stk']

            return df

        # first to run

        # insert "next month closing stock less saftety stock" into 'rssl_less_stk'

        df.loc[7:11,'rssl_less_stk'] = [a_,b_,c_,d_,e_]



        h=6
        df = fillrows(h)

        h=7
        df = fillrows(h)

        h=8
        df = fillrows(h)

        h=9
        df = fillrows(h)

        h=10
        df = fillrows(h)

        h=11
        df = fillrows(h)

        return df
        
    
    total_df = pd.DataFrame() # summary
    total_df1 = pd.DataFrame() # detail
    total_with_OTD = pd.DataFrame() # summary with OTD forecast

    # OTD inputs from dealers
    
    with PdfPages(output_data+'output_plots_'+str(num)+'_'+str(curr_mth())+'.pdf') as pdf:
#     with PdfPages(output_data+'output_plots_'+num+'_'+str(curr_mth())+'.pdf') as pdf:
        
        for i in range(inp.shape[0]):
            first=inp.iloc[i,0]
            second=inp.iloc[i,1]
            third=inp.iloc[i,2]
            fouth=inp.iloc[i,3]
            fifth=inp.iloc[i,4]
            sixth=inp.iloc[i,5]

            leadtime=inp.iloc[i,6] # extract from DPP (pre-determined)

            cls_stk_dpp=inp.iloc[i,7] # extract from DPP
            open_ord_dpp=inp.iloc[i,8] # extract from DPP
            topup = inp.iloc[i,9] # input from CM/dealer
            product_name=inp.iloc[i,11] # extract from DPP
            product_code =inp.iloc[i,10] #--------------------------------------------------------------------
            dealer_name = inp.iloc[i,12] # extract from DPP
            wsp=inp.iloc[i,13] # extract from DPP (pre-determined)
            dlr = inp.iloc[i,12] # extract from DPP (pre-determined)

            # create df dataframe
            df= startfunction()

            a_ = func(df.loc[7,'next_mth_cls_stk'])
            df = startfunction(a_)

            b_ = func(df.loc[8,'next_mth_cls_stk'])
            df = startfunction(a_,b_)

            c_ = func(df.loc[9,'next_mth_cls_stk'])
            df = startfunction(a_,b_,c_)

            d_ = func(df.loc[10, 'next_mth_cls_stk'])
            df = startfunction(a_,b_,c_,d_)

            e_ = func(df.loc[11, 'next_mth_cls_stk'])
            df = startfunction(a_,b_,c_,d_,e_)


            # replace nan with zeros

            df.otd.replace(0,np.nan, inplace=True)
            df.loc[:5,'otd_fcast'].replace(0,np.nan, inplace=True) 
            df.loc[:5,'recom_ord'].replace(0, np.nan, inplace=True)#-----------------------------------------------
            df.fcast_cls_stk.replace(0, np.nan, inplace=True)

            # set negative values to zero

            df.recom_ord = [func(iiii) for iiii in df.recom_ord]

            # adding new columns

        #     df['recom_ord']=df['recom_ord'].fillna(0) #-----------------------------------------------
            df['total_recom_ord']=df['recom_ord']+df['recom_topup']
#             df['total_recom_ord_val']=df['total_recom_ord']*wsp
            df['product_name']=product_name
            df['product_code']=product_code
            df['dealer_name']=dlr

            df2=df.loc[6,['product_code','product_name','recom_safe_stk','fcast_cls_stk','recom_ord','recom_topup',\
                          'total_recom_ord','dealer_name']]
            
            
            df3=df.loc[6,['date','product_code','product_name','recom_safe_stk','fcast_cls_stk','recom_ord','recom_topup',\
                          'total_recom_ord','otd_fcast','dealer_name']]


            df=df.drop(['rssl_less_stk','next_mth_cls_stk','ord_date'], axis=1)

            df=df.iloc[0:12,:]

            df=df.round(0)


            total_df=total_df.append(df2)
            total_with_OTD = total_with_OTD.append(df3)

            total_df=total_df.reset_index(drop=True)
            total_with_OTD=total_with_OTD.reset_index(drop=True)

            total_df1=total_df1.append(df)


            date_ = list(df.loc[6:10,'date'])
            rec_ = list(df.loc[6:10,'total_recom_ord']) #------------------------------------

                        
            with open(output_data+'stocks_log_'+num+'_'+str(curr_mth())+'.txt', "a") as f:
                for d, iii in zip(df.date, df.fcast_cls_stk):
                    if iii <= 0:
#                         print(f"Insufficient stocks on {this_mth} for {product_name} in {dealer_name}")
#                         f.write(f"Insufficient stocks in {this_mth} for {product_name} in {dealer_name}"+ "\n")
                        f.write(f"Insufficient stocks in {this_mth} for {product_name}"+ "\n")


            plt.figure(figsize=(12,8))

            axs=sns.lineplot(data=df, x='date', y='otd')
            axs=sns.lineplot(data=df, x='date', y='otd_fcast')
            axs=sns.lineplot(data=df, x='date', y='total_recom_ord') #------------------------------
            axs=sns.lineplot(data=df, x='date', y='fcast_cls_stk')
            plt.legend(['sell-out (6 months)','sell-out forecast (6 months)','recommended order','forecasted closing stocks'])
            plt.title(f"Forecast Of {product_name} For {dealer_name}", fontsize=16, fontweight='heavy')
            plt.xlabel('Date', fontsize=14)
            plt.ylabel('Quantity', fontsize=14)

            for f, e in enumerate(rec_):
                plt.text(date_[f], rec_[f],e)
#             plt.savefig("forecast_{i}.png".format(i=i))

            pdf.savefig()
            plt.close()
               


    total_df = total_df.rename(columns={'product_code':'Product Code','product_name':'Product Name','recom_safe_stk':'Recommended Safety Stocks',
                                 'fcast_cls_stk':'Forecasted Closing Stocks','recom_ord':'Recommended Order',
                                 'recom_topup':'Recommended Top-up','total_recom_ord':'Total Recommended Order',
                                 'dealer_name':'Dealer Name'})

    total_df1 = total_df1.rename(columns={'cls_stk_dpp':'Closing stocks DPP','open_ord':'Open orders DPP','fcast_open_stk':
                                         'Forecasted opening stocks','fcast_gds_rec':'Forecasted goods received',
                                         'otd_fcast':'OTD Forecast','recom_topup':'Recommended Top-up','recom_safe_stk':
                                         'Recommended Safety Stocks','fcast_cls_stk':'Forecasted Closing Stocks',
                                         'recom_ord':'Recommended Order Qty','total_recom_ord':'Total Recommended Order Qty',
                                         'product_name':'Product Name',
                                         'dealer_name':'Dealer Name','product_code':'Product Code'})





    print()
    print(f"Processed {inp.shape[0]} items for {num}")
    print(f"Historical OTD data : {start} to {last_mth}")
    print(f"Forecast OTD data : {this_mth} to {end_of_forecast}")
    print(f"Processing completed for {num}")
    print()
    
    return total_df, total_df1, total_with_OTD
    

In [5]:
def check_score1(dlr):
    df3 = pd.DataFrame(columns = ['Product Name','Error Rate','Dealer Name'])
    df2 = pd.DataFrame(columns = ['Product Name','Error Rate','Dealer Name'])
    df=b[b['Dealer Name']==dlr]
    pnlist = df['Product Name'].unique()
    pnlist = list(pnlist)
    for pn in pnlist:
        df1=df[df['Product Name']==pn]
        otd = df1.iloc[0:6,6].to_list()
        fcast = df1.iloc[6:12,7].to_list()
        mse = math.sqrt(mean_squared_error(otd,fcast))
        dic = {'Product Name':pn,'Error Rate':mse, 'Dealer Name':dlr}
        df2 = pd.DataFrame(dic, index=[0])
        df3 = pd.concat([df3,df2], axis=0)
    return df3


def curr_mth():
    return datetime.now().strftime('%b%y')
        

# df4 = pd.DataFrame(columns = ['Product Name','Error Rate','Dealer Name'])
# for dlr in total_df1['Dealer Name'].unique():
#     df4 = pd.concat([df4,check_score(dlr)])

# Top 20 products

In [6]:
# load forecast raw data to identify top 20 products per country

path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\forecast function demo\templates\input_data\\'

fd = pd.read_excel(path+'forecast_data.xlsx',header=1)
del fd['Unnamed: 0']

# forward fill country and dealer fields
fd=fd.fillna(method='ffill')

fd.Dealer = [i.replace('/','_') for i in fd.Dealer]

In [7]:
# simplify column names

fd.rename(columns={'COUNTRY_SEA':'country','Dealer':'dealer','PRODUCT CODE':'productcode',
                   'PRODUCT PACKAGING':'product_packaging','Sales_YTD as per Oct':'ytd_sales',
                  '% of Dealer Sales':'percentage_of_sales'},inplace=True)

# change to lower cases
fd.dealer = [i.lower() for i in fd.dealer]

# convert apl promed to promed (dealer)
promed_name = lambda x: 'promed' if x=='apl promed' else x

fd.dealer = [promed_name(i) for i in fd.dealer]

In [8]:
# shortlist top 20 products for Singapore

def top20():
    rev_share_dic={}
    final = pd.DataFrame()
    dlr_list = fd.dealer.unique()
    for c in dlr_list:
        y = fd[fd.dealer==c]
        z = y.sort_values('percentage_of_sales',ascending=False).reset_index(drop=True)
        z['rev_share']=z.percentage_of_sales.cumsum()
        z = z[z.rev_share<=0.8]
        final=pd.concat([final,z])
        
        # index of last row
        try:
            idx = z.index[-1]
            # extract rev share
            rev_share1 = z.loc[idx,'rev_share']
            rev_share_dic[c]=rev_share1
        except:
            continue
    return final[['country','dealer','productcode','product_packaging']], rev_share_dic


In [9]:
# remove certain dealers

x,r = top20()

req = ((x.dealer!='vtech')&(x.dealer!='health promotion board')&(x.dealer!='singhealth')&
      (x.dealer!='quantumleap')&(x.dealer!='bumi')&(x.dealer!='gc ind'))

top20 = x[req]

In [10]:
# top 20 products of 6 major countries

# create a dict of dealer and productcode
top20dict={}
for d,p in zip(top20.dealer,top20.productcode):
    if d in top20dict:
        top20dict[d].append(p)
    else:
        top20dict[d]=[p]
    

# Actual sell out data from DPP

In [11]:
# load sell out data from DPP

sell_out_data_path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\forecast function demo\templates\selloutdata\\'

# sod = pd.read_csv(sell_out_data_path+'DPP Sellout_Cleaned07122022.csv')
# sod = pd.read_excel(sell_out_data_path+'DPP Sellout_Cleaned07122022.xlsx', sheet_name='DPP Sellout_Cleaned07122022')
sod = pd.read_excel(sell_out_data_path+'Sellout as at 17 Feb 2023.xlsx')

In [12]:
# trim columns

sod = sod[['Create Date','Product Code','Product Name','Qty','Created By']]
sod.columns = sod.columns.str.lower()
sod.columns = [i.replace(' ','_') for i in sod.columns]

# change to lower cases
sod.created_by = [i.lower() for i in sod.created_by]

In [13]:
# convert to datetime format
sod.create_date=pd.to_datetime(sod.create_date,format='%d/%m/%Y')

# create new column 'month' to extract month from date
sod['month']=[t.strftime('%Y-%m') for t in sod.create_date]

In [14]:
# remove current in forecast calculation
this_month = datetime.now().strftime('%Y-%m')

req = (sod.month!=this_month)
sod = sod[req]

In [15]:
sod

,create_date,product_code,product_name,qty,created_by,month
0,2021-08-01,0137F097-0020,SOLARE Sculpt Syringe A2 1.9g(1mL),100,sales a: th (achs),2021-08
1,2021-08-01,0138F119-0130,G-aenial Universal Injectable 1.7g JE,50,sales a: th (achs),2021-08
2,2021-08-01,0137F097-0030,SOLARE Sculpt Syringe A3 1.9g(1mL),20,sales a: th (achs),2021-08
3,2021-08-01,0130F129-0010C,Fuji IX 1-1 Package New,10,sales a: th (achs),2021-08
4,2022-04-01,341001,Coe Comfort Professional Package,2,th: accord hs - pennapa (for sellout),2022-04
...,...,...,...,...,...,...
11525,2023-01-01,0131F163-0020,GC Gold Label 9 1-1 Extra A3,480,vn: tuyet hai,2023-01
11526,2023-01-01,0123F100-0000,"FujiCEM Refill Pack, 13.3g Cartridge",190,vn: tuyet hai,2023-01
11527,2023-01-01,0101B888-0000,Fuji Plus 15g Powder,1680,vn: tuyet hai,2023-01
11528,2023-01-01,0101B887-0000,Fuji Plus 1-1 Pkg with Conditioner,2520,vn: tuyet hai,2023-01


In [16]:
# shortlist NK Luck and Pharmaforte

sod_dealer = pd.DataFrame(columns=sod.columns)
c = pd.DataFrame(columns=sod.columns)
c['dealer']=np.nan
c['country']=np.nan

for idx, d in zip(sod.index, sod.created_by):
    if re.search(r'nk luck',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='nk luck'
        c.loc[idx,'country']='singapore'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'pharmaforte',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='pharmaforte'
        c.loc[idx,'country']='singapore'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'phamaforte - ruth dealer',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='pharmaforte'
        c.loc[idx,'country']='singapore'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'cobra',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='cobra'
        c.loc[idx,'country']='indonesia'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'promed',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='promed'
        c.loc[idx,'country']='indonesia'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'ntc',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='ntc'
        c.loc[idx,'country']='malaysia'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'accord',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='accord'
        c.loc[idx,'country']='thailand'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'sales a',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='accord'
        c.loc[idx,'country']='thailand'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'eea',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='eea'
        c.loc[idx,'country']='philippines'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'dental domain',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='dental domain'
        c.loc[idx,'country']='philippines'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'denmedico',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='denmedico'
        c.loc[idx,'country']='vietnam'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'minh phuong',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='minh phuong'
        c.loc[idx,'country']='vietnam'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'edi',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='edi'
        c.loc[idx,'country']='malaysia'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'lien ha',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='lien nha'
        c.loc[idx,'country']='vietnam'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'p2 dental',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='kaldera _ p2 dental'
        c.loc[idx,'country']='malaysia'
        sod_dealer= pd.concat([sod_dealer,c])
    elif re.search(r'tuyet hai',d):
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']='tuyet hai'
        c.loc[idx,'country']='vietnam'
        sod_dealer= pd.concat([sod_dealer,c])
    else:
        c = sod.loc[[idx]]
        c.loc[idx,'dealer']=np.nan
        c.loc[idx,'country']=np.nan
        sod_dealer= pd.concat([sod_dealer,c])
        
sod_dealer.dropna(inplace=True)

# list of dealers
dlr_list = sod_dealer.dealer.unique()

# Create function to determine the name of months needed for forecast

In [17]:
# calculate the specific months needed for forecast
def current_month():
    return int(datetime.now().strftime('%m'))


In [18]:
import pandas as pd
from datetime import datetime

now = datetime.now()
end_date = (now - pd.DateOffset(months=1)).replace(day=1)
start_date = (end_date - pd.DateOffset(months=5)).replace(day=1)
dates = pd.date_range(start=start_date, end=end_date, freq='MS').strftime('%Y-%m').tolist()

print(dates)

['2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01']


In [19]:
# create function to determine the number of months whether it qualifies for 6 months of historical data
def num_of_months(dlr,pc):
    
    x = sod_dealer[sod_dealer.dealer==dlr]
    y = x[x.product_code==pc]
    z = y.month.unique()

    num_of_months = len(set(z)&set(dates))
    months_item = set(z)&set(dates)

    return num_of_months

# Load inventory from DPP

In [20]:
inventory_path = r'C:\Users\stephenl\OneDrive - GC International AG\ML\forecast function demo\templates\inventory\\'

inv_folder = os.listdir(inventory_path)
excel_files = [invi for invi in inv_folder if invi.endswith(".csv")]
excel_files = sorted(excel_files, key=lambda x: os.path.getmtime(os.path.join(inventory_path, x)), reverse=True)
latest_file_path = os.path.join(inventory_path, excel_files[0])

if re.search(r'inventory', latest_file_path):
    inv = pd.read_csv(latest_file_path)
    inv.dealer = inv.dealer.str.lower()
else:
    print("File not found.")


In [21]:
# create new column 'dealer1'

# shortlist NK Luck and Pharmaforte

# sod_dealer = pd.DataFrame(columns=sod.columns)
# c = pd.DataFrame(columns=sod.columns)
inv['dealer1']=np.nan
inv['country']=np.nan

for idx, d in zip(inv.index, inv.dealer):
    if re.search(r'nk luck',d):
        inv.loc[idx,'dealer1']='nk luck'
        inv.loc[idx,'country']='singapore'

    elif re.search(r'pharmaforte',d):
        inv.loc[idx,'dealer1']='pharmaforte'
        inv.loc[idx,'country']='singapore'

    elif re.search(r'cobra',d):
        inv.loc[idx,'dealer1']='cobra'
        inv.loc[idx,'country']='indonesia'

    elif re.search(r'promed',d):
        inv.loc[idx,'dealer1']='promed'
        inv.loc[idx,'country']='indonesia'

    elif re.search(r'ntc',d):
        inv.loc[idx,'dealer1']='ntc'
        inv.loc[idx,'country']='malaysia'

    elif re.search(r'accord',d):
        inv.loc[idx,'dealer1']='accord'
        inv.loc[idx,'country']='thailand'

    elif re.search(r'eea',d):
        inv.loc[idx,'dealer1']='eea'
        inv.loc[idx,'country']='philippines'

    elif re.search(r'dental domain',d):
        inv.loc[idx,'dealer1']='dental domain'
        inv.loc[idx,'country']='philippines'

    elif re.search(r'denmedico',d):
        inv.loc[idx,'dealer1']='denmedico'
        inv.loc[idx,'country']='vietnam'

    elif re.search(r'minh phuong',d):
        inv.loc[idx,'dealer1']='minh phuong'
        inv.loc[idx,'country']='vietnam'

    elif re.search(r'edi',d):
        inv.loc[idx,'dealer1']='edi'
        inv.loc[idx,'country']='malaysia'

    elif re.search(r'lien',d):
        inv.loc[idx,'dealer1']='lien nha'
        inv.loc[idx,'country']='vietnam'

    elif re.search(r'p2 dental',d) or re.search(r'kaldera',d):
        inv.loc[idx,'dealer1']='kaldera _ p2 dental'
        inv.loc[idx,'country']='malaysia'

    elif re.search(r'tuyet hai',d):
        inv.loc[idx,'dealer1']='tuyet hai'
        inv.loc[idx,'country']='vietnam'

    else:
        inv.loc[idx,'dealer1']=np.nan
        inv.loc[idx,'country']=np.nan

        
inv.dropna(inplace=True)

In [22]:
# consolidate inventory status using groupby

inv1 = inv.groupby(['dealer1','code','packaging']).agg(total_openorder=('open_order','sum'),total_available=('available','sum')).reset_index()

# Create function to convert array to spot value

In [23]:
# convert array to integer or string
def spot(x):
    for i in x:
        return i

# Input data into input template

In [24]:
sod_dealer

,create_date,product_code,product_name,qty,created_by,month,dealer,country
0,2021-08-01,0137F097-0020,SOLARE Sculpt Syringe A2 1.9g(1mL),100,sales a: th (achs),2021-08,accord,thailand
1,2021-08-01,0138F119-0130,G-aenial Universal Injectable 1.7g JE,50,sales a: th (achs),2021-08,accord,thailand
2,2021-08-01,0137F097-0030,SOLARE Sculpt Syringe A3 1.9g(1mL),20,sales a: th (achs),2021-08,accord,thailand
3,2021-08-01,0130F129-0010C,Fuji IX 1-1 Package New,10,sales a: th (achs),2021-08,accord,thailand
4,2022-04-01,341001,Coe Comfort Professional Package,2,th: accord hs - pennapa (for sellout),2022-04,accord,thailand
...,...,...,...,...,...,...,...,...
11525,2023-01-01,0131F163-0020,GC Gold Label 9 1-1 Extra A3,480,vn: tuyet hai,2023-01,tuyet hai,vietnam
11526,2023-01-01,0123F100-0000,"FujiCEM Refill Pack, 13.3g Cartridge",190,vn: tuyet hai,2023-01,tuyet hai,vietnam
11527,2023-01-01,0101B888-0000,Fuji Plus 15g Powder,1680,vn: tuyet hai,2023-01,tuyet hai,vietnam
11528,2023-01-01,0101B887-0000,Fuji Plus 1-1 Pkg with Conditioner,2520,vn: tuyet hai,2023-01,tuyet hai,vietnam


In [25]:
# extract productcodes from top20 products



input_temp = pd.DataFrame(columns=['first','second','third','fourth','fifth','sixth','leadtime','closing_stks_dpp',
                                  'open_order_dpp','topup','product_code','product_name','dealer_name','wholesale_px'])
# start row counter
n=0

# append qualified dealer
dlr_qualified=[]
for dlr in dlr_list:
    for pc in top20dict[dlr]:

        if num_of_months(dlr,pc)==6:
            x = sod_dealer[sod_dealer.dealer==dlr]
            
            # append dealers
            dlr_qualified.append(dlr)
            
            p = x[x.product_code==pc]
            g = p.groupby('month').agg(total_qty=('qty','sum'))
            s = g.sort_index(ascending=False).head(6)
            input_temp.loc[n,'first']=s.iloc[5,0]
            input_temp.loc[n,'second']=s.iloc[4,0]
            input_temp.loc[n,'third']=s.iloc[3,0]
            input_temp.loc[n,'fourth']=s.iloc[2,0]
            input_temp.loc[n,'fifth']=s.iloc[1,0]
            input_temp.loc[n,'sixth']=s.iloc[0,0]
            input_temp.loc[n,'leadtime']=2
            inv1 = (inv.groupby(['dealer1','code','packaging']).agg(total_openorder=('open_order','sum'),
                                                        total_available=('available','sum')).reset_index())
            inv2 = inv1[inv1.dealer1==dlr]
            inv3 = spot(inv2[inv2.code==pc].total_available)
            input_temp.loc[n,'closing_stks_dpp']=inv3
            inv4 = spot(inv2[inv2.code==pc].total_openorder)
            input_temp.loc[n,'open_order_dpp']=inv4
            input_temp.loc[n,'topup']=0
            input_temp.loc[n,'product_code']=pc
            inv5 = spot(inv2[inv2.code==pc].packaging)
            input_temp.loc[n,'product_name']=inv5
            input_temp.loc[n,'dealer_name']=dlr
            input_temp.loc[n,'wholesale_px']=0
            n+=1
            
            
            
#             print(f"dealer: {dlr}, productcode: {pc}, qualify: {True}")
unique_dlr_list=[]         
for i in dlr_qualified:
    if i not in unique_dlr_list:
        unique_dlr_list.append(i)
        
unique_dlr_list=[i.upper() for i in unique_dlr_list]

In [26]:
dealer_list = input_temp.dealer_name.unique()
dealer_list = [i.replace('/','_') for i in dealer_list]

In [27]:
# remove all text file in output_data_folder
output_data_folder = os.listdir(output_data)
for logfile in output_data_folder:
    if logfile.endswith('.txt'):
        path = os.path.join(output_data, logfile)
        os.remove(path)


# create error rate dataframe
df5 = pd.DataFrame(columns = ['Product Name','Error Rate','Dealer Name'])
df6 = pd.DataFrame()

# output files
for d in dealer_list:
    inp = input_temp[input_temp.dealer_name==d]
    a,b,c = forecast6(d)
    a.to_csv(output_data+'summary_'+d+'_'+str(curr_mth())+'.csv',index=False)
    b.to_csv(output_data+'details_'+d+'_'+str(curr_mth())+'.csv',index=False)
    c.to_csv(output_data+'summary_w_OTD_'+d+'_'+str(curr_mth())+'.csv',index=False)
    
    # create excel sheet to show the total share of revenue for forecasted products
    rev_share2 = a.merge(fd,left_on=['Dealer Name','Product Code'], right_on=['dealer','productcode'])
    rev_share3=rev_share2.groupby('dealer').agg(total_rev_share=('percentage_of_sales','sum'))
    rev_share3=rev_share3.reset_index()
    rev_share3=rev_share3.round(2)
#     rev_share3.to_csv(output_data+'revenue_share_of_products_'+d+'_'+str(curr_mth())+'.csv',index=False)
    df6 = pd.concat([df6,rev_share3])
    
    # error
    df5 = pd.concat([df5,check_score1(d)])
    
df6.to_csv(output_data+'revenue_share_of products_'+str(curr_mth())+'.csv',index=False)
    
df5.to_csv(output_data+'accuracy_'+str(curr_mth())+'.csv',index=False)
    


Processed 22 items for accord
Historical OTD data : August-2022 to January-2023
Forecast OTD data : February-2023 to July-2023
Processing completed for accord


Processed 24 items for cobra
Historical OTD data : August-2022 to January-2023
Forecast OTD data : February-2023 to July-2023
Processing completed for cobra


Processed 16 items for nk luck
Historical OTD data : August-2022 to January-2023
Forecast OTD data : February-2023 to July-2023
Processing completed for nk luck


Processed 19 items for ntc
Historical OTD data : August-2022 to January-2023
Forecast OTD data : February-2023 to July-2023
Processing completed for ntc


Processed 6 items for kaldera _ p2 dental
Historical OTD data : August-2022 to January-2023
Forecast OTD data : February-2023 to July-2023
Processing completed for kaldera _ p2 dental


Processed 16 items for pharmaforte
Historical OTD data : August-2022 to January-2023
Forecast OTD data : February-2023 to July-2023
Processing completed for pharmaforte

